Epicurious, Part 2: Once-per-row scraping
Then, open your search results csv, filter for ONLY recipes. Merge the following fields with your original recipes and save as a new CSV file:

--Ingredients

--Directions

--Tags

Tip: If you use .find for your directions/ingredients, it'll print them all on one line. But if you use .find_all to separate them, it makes your life a lot harder! ...unless you just steal this code:

# Get the text from each step individually

# Then join them together with \n between each step

'\n'.join([step.text for step in steps])

In [65]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [66]:
df = pd.read_csv("Epicurious_part1.csv")
df = df.drop(columns=['make_again', 'summary', 'rating'])
df = df[df.category == 'recipe']

In [67]:
df.head()

,category,link,name
0,recipe,/recipes/food/views/spicy-lightly-pickled-cucu...,Spicy Lightly Pickled Cucumbers
1,recipe,/recipes/food/views/cucumbers-with-ajo-blanco-...,Cucumbers with Ajo Blanco Sauce
2,recipe,/recipes/food/views/cold-beef-tenderloin-with-...,Cold Beef Tenderloin with Tomatoes and Cucumbers
3,recipe,/recipes/food/views/fried-fish-sandwiches-with...,Fried Fish Sandwiches with Cucumbers and Tarta...
4,recipe,/recipes/food/views/farro-spelt-grain-salad-wi...,Grain Salad with Tomatoes and Cucumbers


In [68]:
url = "https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers"
response = requests.get(url)
doc = BeautifulSoup(response.text)

In [69]:
ingredients = [step.text.strip() for step in doc.find_all(class_="ingredient")]
ingredients

['2 lb. medium Persian cucumbers (about 12), cut lengthwise into spears',
 '1/4 cup white wine vinegar or unseasoned rice vinegar',
 '1 tsp. sugar',
 '3/4 tsp. crushed red pepper flakes',
 '2 tsp. kosher salt, plus more',
 '2 Tbsp. chopped dill',
 '2 Tbsp. fresh lemon juice']

In [70]:
[step.text.strip() for step in doc.find_all(class_="preparation-step")]

['Toss cucumbers in a large bowl with vinegar, sugar, red pepper flakes, and 2 tsp. salt. Chill, tossing once, at least 1 hour and up to 6 hours.',
 'Just before serving, add dill and lemon juice and toss to combine. Taste and season with more salt if needed.']

In [71]:
tags = [step.text.strip() for step in doc.find(class_="tags").find_all('dt')]
tags

['Bon Appétit',
 'Pickles',
 "Hors D'Oeuvre",
 'Appetizer',
 'Side',
 "New Year's Eve",
 'Cocktail Party',
 'Cucumber',
 'Dill',
 'Vinegar',
 'Summer']

In [72]:
url = "https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers"
print("We are scraping", url)

response = requests.get(url)
doc = BeautifulSoup(response.text)

page = {}

page['ingredients'] = [step.text for step in doc.find_all(class_="ingredient")]
page['Preparation'] = [step.text.strip() for step in doc.find_all(class_="preparation-step")]
page['tags'] = [step.text for step in doc.find(class_="tags").find_all('dt')]

print(page)

We are scraping https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers
{'ingredients': ['2 lb. medium Persian cucumbers (about 12), cut lengthwise into spears', '1/4 cup white wine vinegar or unseasoned rice vinegar', '1 tsp. sugar', '3/4 tsp. crushed red pepper flakes', '2 tsp. kosher salt, plus more', '2 Tbsp. chopped dill', '2 Tbsp. fresh lemon juice'], 'Preparation': ['Toss cucumbers in a large bowl with vinegar, sugar, red pepper flakes, and 2 tsp. salt. Chill, tossing once, at least 1 hour and up to 6 hours.', 'Just before serving, add dill and lemon juice and toss to combine. Taste and season with more salt if needed.'], 'tags': ['Bon Appétit', 'Pickles', "Hors D'Oeuvre", 'Appetizer', 'Side', "New Year's Eve", 'Cocktail Party', 'Cucumber', 'Dill', 'Vinegar', 'Summer']}


In [73]:
df.shape

(108, 3)

In [74]:
def scrape_page(row):
    url = f"https://www.epicurious.com{row['link']}"
    print("Scraping", url)

In [75]:
df.head().apply(scrape_page, axis=1)

Scraping https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/cucumbers-with-ajo-blanco-sauce
Scraping https://www.epicurious.com/recipes/food/views/cold-beef-tenderloin-with-tomatoes-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/fried-fish-sandwiches-with-cucumbers-and-tartar-sauce
Scraping https://www.epicurious.com/recipes/food/views/farro-spelt-grain-salad-with-tomatoes-and-cucumbers


0    None
1    None
2    None
3    None
4    None
dtype: object

In [76]:
def scrape_page(row):
    url = f"https://www.epicurious.com{row['link']}"
    print("Scraping", url)

    response = requests.get(url)
    doc = BeautifulSoup(response.text)


    page = {}
    
    try:
        page['Ingredients'] = [step.text for step in doc.find_all(class_="ingredient")]
    except:
        "ingredients not available"
    try:
        page['Preparation'] = [step.text.strip() for step in doc.find_all(class_="preparation-step")]
    except:
        "preparation not available"
    try:
        page['Tags'] = [step.text for step in doc.find(class_="tags").find_all('dt')]
    except:
        "no tags"
        
    return pd.Series(page)

In [77]:
scrape_page(df.loc[0])

Scraping https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers


Ingredients    [2 lb. medium Persian cucumbers (about 12), cu...
Preparation    [Toss cucumbers in a large bowl with vinegar, ...
Tags           [Bon Appétit, Pickles, Hors D'Oeuvre, Appetize...
dtype: object

In [78]:
scrape_page(df.loc[1])

Scraping https://www.epicurious.com/recipes/food/views/cucumbers-with-ajo-blanco-sauce


Ingredients    [5 mini seedless or Persian cucumbers (about 1...
Preparation    [Preheat oven to 350°F. Cut cucumbers about 1"...
Tags           [Bon Appétit, Sauce, Condiment, Garlic, Almond...
dtype: object

In [79]:
scraped_df = df.apply(scrape_page, axis=1)
scraped_df

Scraping https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/spicy-lightly-pickled-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/cucumbers-with-ajo-blanco-sauce
Scraping https://www.epicurious.com/recipes/food/views/cold-beef-tenderloin-with-tomatoes-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/fried-fish-sandwiches-with-cucumbers-and-tartar-sauce
Scraping https://www.epicurious.com/recipes/food/views/farro-spelt-grain-salad-with-tomatoes-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/grilled-charred-cabbage-with-goat-cheese-raita-and-cucumbers
Scraping https://www.epicurious.com/recipes/food/views/cucumber-seaweed-salad
Scraping https://www.epicurious.com/recipes/food/views/pineapple-shrimp-noodle-bowls
Scraping https://www.epicurious.com/recipes/food/views/sambal-shrimp-lettuce-wraps
Scraping https://www.epicurious.com/recipes/food/views/sm

Scraping https://www.epicurious.com/recipes/food/views/crunchy-chicken-salad-with-shiitake-mushrooms-spicy-sesame-dressing-and-peanuts-101251
Scraping https://www.epicurious.com/recipes/food/views/sauteed-cucumbers-with-cumin-and-mint-12413
Scraping https://www.epicurious.com/recipes/food/views/sweet-and-sour-cucumbers-with-fresh-dill-109713
Scraping https://www.epicurious.com/recipes/food/views/green-papaya-salad-with-shrimp-10520
Scraping https://www.epicurious.com/recipes/food/views/southeast-asian-style-turkey-burgers-with-pickled-cucumbers-101071
Scraping https://www.epicurious.com/recipes/food/views/basil-tabbouleh-12521
Scraping https://www.epicurious.com/recipes/food/views/african-adobo-rubbed-tuna-steaks-108201
Scraping https://www.epicurious.com/recipes/food/views/cordoban-gazpacho-106875
Scraping https://www.epicurious.com/recipes/food/views/red-flannel-hash-with-fried-eggs-5824
Scraping https://www.epicurious.com/recipes/food/views/creamy-cucumbers-108996
Scraping https://w

,Ingredients,Preparation,Tags
0,"[2 lb. medium Persian cucumbers (about 12), cu...","[Toss cucumbers in a large bowl with vinegar, ...","[Bon Appétit, Pickles, Hors D'Oeuvre, Appetize..."
1,[5 mini seedless or Persian cucumbers (about 1...,"[Preheat oven to 350°F. Cut cucumbers about 1""...","[Bon Appétit, Sauce, Condiment, Garlic, Almond..."
2,"[1/4 cup extra-virgin olive oil, plus more, 1 ...",[Prepare a grill for 2-zone heat (for a charco...,"[Bon Appétit, Dinner, Beef Tenderloin, Beef, T..."
3,"[1/2 cup mayonnaise, 1/4 cup finely grated lem...","[Mix mayonnaise, lemon zest and juice, relish,...","[Bon Appétit, Sandwich, Fish, Seafood, Fry, Be..."
4,"[2 cups semi-pearled farro or spelt, Kosher sa...",[Preheat oven to 350°F. Cook farro in a large ...,"[Bon Appétit, Salad, Side, Grains, Cucumber, T..."
5,"[1 garlic clove, smashed, 4 oz. fresh goat che...","[Pulse garlic, goat cheese, yogurt, 1 cup mint...","[Bon Appétit, Cabbage, Vegetarian, Grill/Barbe..."
6,"[3/4 ounce dried wakame seaweed, cut into 1-in...","[In a bowl, soak the wakame in warm water for ...","[HarperCollins, Salad, Side, Cucumber, Ginger,..."
7,"[12 oz. pad Thai–style rice noodles, 1 lb. lar...",[Cook rice noodles according to package direct...,"[Shrimp, Noodle, Pineapple, Soy Sauce, Ginger,..."
8,[1/2 cup hot chili paste (such as sambal oelek...,"[Whisk hot chili paste, honey, vinegar, and 2 ...","[Bon Appétit, Dinner, Seafood, Shellfish, Shri..."
9,[5 mini seedless or Persian cucumbers or 1 sma...,[Gently smash cucumbers with a rolling pin or ...,"[Bon Appétit, Salad, Cucumber, Side, Lemon, Qu..."


In [80]:
df.head()

,category,link,name
0,recipe,/recipes/food/views/spicy-lightly-pickled-cucu...,Spicy Lightly Pickled Cucumbers
1,recipe,/recipes/food/views/cucumbers-with-ajo-blanco-...,Cucumbers with Ajo Blanco Sauce
2,recipe,/recipes/food/views/cold-beef-tenderloin-with-...,Cold Beef Tenderloin with Tomatoes and Cucumbers
3,recipe,/recipes/food/views/fried-fish-sandwiches-with...,Fried Fish Sandwiches with Cucumbers and Tarta...
4,recipe,/recipes/food/views/farro-spelt-grain-salad-wi...,Grain Salad with Tomatoes and Cucumbers


In [81]:
df = df.merge(scraped_df, left_index=True, right_index=True)
df

,category,link,name,Ingredients,Preparation,Tags
0,recipe,/recipes/food/views/spicy-lightly-pickled-cucu...,Spicy Lightly Pickled Cucumbers,"[2 lb. medium Persian cucumbers (about 12), cu...","[Toss cucumbers in a large bowl with vinegar, ...","[Bon Appétit, Pickles, Hors D'Oeuvre, Appetize..."
1,recipe,/recipes/food/views/cucumbers-with-ajo-blanco-...,Cucumbers with Ajo Blanco Sauce,[5 mini seedless or Persian cucumbers (about 1...,"[Preheat oven to 350°F. Cut cucumbers about 1""...","[Bon Appétit, Sauce, Condiment, Garlic, Almond..."
2,recipe,/recipes/food/views/cold-beef-tenderloin-with-...,Cold Beef Tenderloin with Tomatoes and Cucumbers,"[1/4 cup extra-virgin olive oil, plus more, 1 ...",[Prepare a grill for 2-zone heat (for a charco...,"[Bon Appétit, Dinner, Beef Tenderloin, Beef, T..."
3,recipe,/recipes/food/views/fried-fish-sandwiches-with...,Fried Fish Sandwiches with Cucumbers and Tarta...,"[1/2 cup mayonnaise, 1/4 cup finely grated lem...","[Mix mayonnaise, lemon zest and juice, relish,...","[Bon Appétit, Sandwich, Fish, Seafood, Fry, Be..."
4,recipe,/recipes/food/views/farro-spelt-grain-salad-wi...,Grain Salad with Tomatoes and Cucumbers,"[2 cups semi-pearled farro or spelt, Kosher sa...",[Preheat oven to 350°F. Cook farro in a large ...,"[Bon Appétit, Salad, Side, Grains, Cucumber, T..."
5,recipe,/recipes/food/views/grilled-charred-cabbage-wi...,Charred Cabbage with Goat Cheese Raita and Cuc...,"[1 garlic clove, smashed, 4 oz. fresh goat che...","[Pulse garlic, goat cheese, yogurt, 1 cup mint...","[Bon Appétit, Cabbage, Vegetarian, Grill/Barbe..."
6,recipe,/recipes/food/views/cucumber-seaweed-salad,Cucumber-Seaweed Salad,"[3/4 ounce dried wakame seaweed, cut into 1-in...","[In a bowl, soak the wakame in warm water for ...","[HarperCollins, Salad, Side, Cucumber, Ginger,..."
7,recipe,/recipes/food/views/pineapple-shrimp-noodle-bowls,Pineapple Shrimp Noodle Bowls,"[12 oz. pad Thai–style rice noodles, 1 lb. lar...",[Cook rice noodles according to package direct...,"[Shrimp, Noodle, Pineapple, Soy Sauce, Ginger,..."
8,recipe,/recipes/food/views/sambal-shrimp-lettuce-wraps,Sambal Shrimp Lettuce Wraps,[1/2 cup hot chili paste (such as sambal oelek...,"[Whisk hot chili paste, honey, vinegar, and 2 ...","[Bon Appétit, Dinner, Seafood, Shellfish, Shri..."
9,recipe,/recipes/food/views/smashed-cucumber-salad-wit...,Smashed Cucumber Salad with Lemon and Celery Salt,[5 mini seedless or Persian cucumbers or 1 sma...,[Gently smash cucumbers with a rolling pin or ...,"[Bon Appétit, Salad, Cucumber, Side, Lemon, Qu..."


In [82]:
df.to_csv("Epicurious_part2.csv", index=False)